# Reflexion 开发指南

本指南介绍了如何使用 LangGraph 构建具有自我反思能力的代理，该代理基于 Shinn 等人的 Reflexion 架构。该代理通过自我批判其任务响应，以生成更高质量的最终答案，代价是增加了执行时间。

## 工作流程

1. 安装与设置
2. 设置 LangSmith 用于 LangGraph 开发
3. 定义我们的 LLM
4. Actor（具有反思功能）
5. 构建工具
6. 初始响应器
7. 修订
8. 创建工具节点
9. 构建图形

## 1. 安装与设置

首先，安装 `langgraph`（用于框架）、`langchain_anthropic`（用于 LLM），以及 `tavily-python`（用于搜索引擎）。

In [ ]:
%pip install -U --quiet langgraph langchain_anthropic
%pip install -U --quiet tavily-python
import getpass
import os

def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)

_set_if_undefined("ANTHROPIC_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

我们将使用 Tavily Search 作为工具。你可以在 [这里](https://www.tavily.com/) 获取 API 密钥，或者替换为你选择的其他工具。

## 2. 设置 LangSmith 用于 LangGraph 开发

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许你使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序——阅读更多关于如何开始的信息 [here](https://smith.langchain.com/)。

## 3. 定义我们的 LLM

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-turbo-preview")

# 你也可以使用其他提供商
# from langchain_anthropic import ChatAnthropic

# llm = ChatAnthropic(model="claude-3-sonnet-20240229")

## 4. Actor（具有反思功能）

Reflexion 的主要组件是 "actor"，这是一个通过自我批判其响应并根据自我批判重新执行以改进的代理。其主要子组件包括：

- 工具/工具执行
- 初始响应器：生成任务的初始响应（和自我反思）
- 修订者：基于先前的反思重新响应（和反思）

我们首先定义工具执行上下文。

--------------------

## 5. 构建工具

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

## 6. 初始响应器

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import ValidationError
# 注意：你必须使用 langchain-core >= 0.3 与 Pydantic v2
from pydantic import BaseModel, Field

class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")

class AnswerQuestion(BaseModel):
    answer: str = Field(description="~250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    search_queries: list[str] = Field(
        description="1-3 search queries for researching improvements to address the critique of your current answer."
    )

class ResponderWithRetries:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    def respond(self, state: list):
        response = []
        for attempt in range(3):
            response = self.runnable.invoke(
                {"messages": state}, {"tags": [f"attempt:{attempt}"]}
            )
            try:
                self.validator.invoke(response)
                return response
            except ValidationError as e:
                state = state + [
                    response,
                    ToolMessage(
                        content=f"{repr(e)}

Pay close attention to the function schema.

"
                        + self.validator.schema_json()
                        + " Respond by fixing all validation errors.",
                        tool_call_id=response.tool_calls[0]["id"],
                    ),
                ]
        return response

## 7. 修订

In [ ]:
revise_instructions = '''Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
'''

扩展初始答案的模式以包含引用。强制模型引用可以鼓励扎实的响应。

In [ ]:
# 扩展初始答案的模式以包含引用。
# 强制引用可以鼓励模型生成有根据的响应
class ReviseAnswer(AnswerQuestion):
    references: list[str] = Field(description="Citations motivating your updated answer.")

## 8. 创建工具节点

In [ ]:
from langchain_core.tools import StructuredTool
from langgraph.prebuilt import ToolNode

def run_queries(search_queries: list[str], **kwargs):
    return tavily_tool.batch([{"query": query} for query in search_queries])

tool_node = ToolNode(
    [
        StructuredTool.from_function(run_queries, name=AnswerQuestion.__name__),
        StructuredTool.from_function(run_queries, name=ReviseAnswer.__name__),
    ]
)

## 9. 构建图形

In [ ]:
from typing import Literal
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list, add_messages]

MAX_ITERATIONS = 5
builder = StateGraph(State)
builder.add_node("draft", first_responder.respond)

builder.add_node("execute_tools", tool_node)
builder.add_node("revise", revisor.respond)
builder.add_edge("draft", "execute_tools")
builder.add_edge("execute_tools", "revise")

def _get_num_iterations(state: list):
    i = 0
    for m in state[::-1]:
        if m.type not in {"tool", "ai"}:
            break
        i += 1
    return i

def event_loop(state: list) -> Literal["execute_tools", "__end__"]:
    num_iterations = _get_num_iterations(state)
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"

builder.add_conditional_edges("revise", event_loop)
builder.add_edge(START, "draft")
graph = builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass